In [1]:
!pip install -q groq PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00


In [19]:
import os
import json
from groq import Groq
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
client = Groq(api_key=os.environ["GROQ_API_KEY"])

def call_llm(prompt: str) -> str:
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama-3.1-8b-instant",  # ✅ Updated model name
        temperature=0.0
    )
    return response.choices[0].message.content

In [14]:
import PyPDF2

def parse_resume(file_path: str) -> str:
    try:
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            return "\n".join(page.extract_text() or "" for page in reader.pages)
    except Exception as e:
        return f"ERROR_PARSING_RESUME: {str(e)}"

In [15]:
def resume_parser_agent(resume_text: str) -> dict:
    if "ERROR_PARSING_RESUME" in resume_text:
        return {"candidate_profile": {}, "parsing_error": True}

    prompt = f"""
Extract structured info from this resume. Return ONLY JSON with these keys:
{{"name": "...", "years_experience": 0, "skills": ["..."], "education": "...", "domain_experience": ["..."]}}

Resume:
{resume_text}
"""
    try:
        text = call_llm(prompt)
        print("📝 Resume LLM Response:", text[:300])  # DEBUG

        # Extract JSON robustly
        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1:
            text = text[start:end+1]

        # Clean common issues
        text = text.replace("\n", "").replace("\\", "").strip()
        profile = json.loads(text)
        return {"candidate_profile": profile, "parsing_error": False}
    except Exception as e:
        print("❌ Resume Parsing Error:", e)
        # Fallback structured data
        return {"candidate_profile": {
            "name": "Unknown",
            "years_experience": 0,
            "skills": [],
            "education": "Not extracted",
            "domain_experience": []
        }, "parsing_error": False}  # Continue processing

In [16]:
def jd_parser_agent(jd_text: str) -> dict:
    prompt = f"""
Extract job requirements. Return ONLY JSON with these keys:
{{"role": "...", "required_skills": ["..."], "required_experience_years": 0, "strictness_level": "strict"|"flexible"|"vague"}}

Job Description:
{jd_text}
"""
    try:
        text = call_llm(prompt)
        print("📝 JD LLM Response:", text[:300])  # DEBUG

        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1:
            text = text[start:end+1]

        text = text.replace("\n", "").replace("\\", "").strip()
        reqs = json.loads(text)
        return {"job_requirements": reqs}
    except Exception as e:
        print("❌ JD Parsing Error:", e)
        return {"job_requirements": {
            "role": "Unknown",
            "required_skills": [],
            "required_experience_years": 0,
            "strictness_level": "vague"
        }}

In [17]:
def match_agent(state: dict) -> dict:
    profile = state.get("candidate_profile", {})
    job = state.get("job_requirements", {})

    # Skill matching
    cand_skills = set(s.lower() for s in profile.get("skills", []))
    req_skills = set(s.lower() for s in job.get("required_skills", []))
    matched = cand_skills & req_skills
    skill_score = len(matched) / max(1, len(req_skills))

    # Experience matching
    cand_exp = profile.get("years_experience", 0)
    req_exp = job.get("required_experience_years", 0)
    exp_score = min(cand_exp / req_exp, 1.0) if req_exp > 0 else 0.7

    match_score = round(0.6 * skill_score + 0.4 * exp_score, 2)
    strictness = job.get("strictness_level", "flexible")

    # Decision logic
    if strictness == "vague":
        requires_human, recommendation, confidence = True, "Needs manual review", 0.7
    elif strictness == "strict":
        if match_score >= 0.85:
            requires_human, recommendation, confidence = False, "Proceed to interview", 0.95
        elif match_score >= 0.3:
            requires_human, recommendation, confidence = True, "Needs manual review", 0.75
        else:
            requires_human, recommendation, confidence = False, "Reject", 0.90
    else:  # flexible
        if match_score >= 0.75:
            requires_human, recommendation, confidence = False, "Proceed to interview", 0.90
        elif match_score >= 0.5:
            requires_human, recommendation, confidence = True, "Needs manual review", 0.70
        else:
            requires_human, recommendation, confidence = False, "Reject", 0.85

    reasoning = (
        f"Match score {match_score}. Matched skills: {list(matched)}. "
        f"Experience: {cand_exp}y vs {req_exp}y required. Strictness: {strictness}."
    )

    return {
        "match_score": match_score,
        "confidence": confidence,
        "requires_human": requires_human,
        "recommendation": recommendation,
        "reasoning_summary": reasoning
    }

In [20]:
from google.colab import files

def validate_output(resume_name: str, jd_name: str, result: dict):
    print(f"\n🔍 VALIDATION: {resume_name} + {jd_name}")
    print(f"✅ Match Score: {result['match_score']}")
    print(f"✅ Requires Human: {result['requires_human']}")
    print(f"✅ Reasoning: {result['reasoning_summary'][:120]}...")

    if "vague" in jd_name and not result["requires_human"]:
        print("❌ FAILED: Vague JD must require human review!")
    else:
        print("✅ PASSED all validation checks")

# Upload files
print("📤 Upload resume (PDF):")
resume_file = list(files.upload().keys())[0]
print("📄 Upload job description (TXT):")
jd_file = list(files.upload().keys())[0]

# Agentic flow
state = {}
state.update(resume_parser_agent(parse_resume(resume_file)))

if not state.get("parsing_error"):
    with open(jd_file, 'r') as f:
        state.update(jd_parser_agent(f.read()))

result = match_agent(state)

final_output = {
    "match_score": float(result["match_score"]),
    "recommendation": result["recommendation"],
    "requires_human": bool(result["requires_human"]),
    "confidence": float(result["confidence"]),
    "reasoning_summary": result["reasoning_summary"]
}

validate_output(resume_file, jd_file, final_output)
print("\n" + "="*50)
print("FINAL OUTPUT (Pitcrew Format):")
print(json.dumps(final_output, indent=2))

📤 Upload resume (PDF):


Saving resume_02_rahul_verma.pdf to resume_02_rahul_verma (1).pdf
📄 Upload job description (TXT):


Saving jd_02_senior_fintech_strict.txt to jd_02_senior_fintech_strict (1).txt
📝 Resume LLM Response: ```json
{
  "name": "Rahul Verma",
  "years_experience": 2,
  "skills": [
    "Python",
    "SQL",
    "R",
    "Pandas",
    "NumPy",
    "Matplotlib",
    "Seaborn",
    "Jupyter",
    "PostgreSQL",
    "MySQL",
    "SQLite",
    "Tableau",
    "Power BI",
    "Excel",
    "Flask",
    "Django",
 
📝 JD LLM Response: ```json
{
  "role": "Senior Backend Engineer - Fintech",
  "required_skills": [
    "Python",
    "Django or FastAPI",
    "PostgreSQL",
    "Redis/Caching strategies",
    "Docker & Kubernetes",
    "Message queues (Kafka/RabbitMQ)",
    "AWS services (EC2, RDS, SQS, Lambda)"
  ],
  "required_exper

🔍 VALIDATION: resume_02_rahul_verma (1).pdf + jd_02_senior_fintech_strict (1).txt
✅ Match Score: 0.37
✅ Requires Human: True
✅ Reasoning: Match score 0.37. Matched skills: ['postgresql', 'python']. Experience: 2y vs 4y required. Strictness: strict....
✅ PASSED all validation c

In [21]:
from google.colab import files

def validate_output(resume_name: str, jd_name: str, result: dict):
    print(f"\n🔍 VALIDATION: {resume_name} + {jd_name}")
    print(f"✅ Match Score: {result['match_score']}")
    print(f"✅ Requires Human: {result['requires_human']}")
    print(f"✅ Reasoning: {result['reasoning_summary'][:120]}...")

    if "vague" in jd_name and not result["requires_human"]:
        print("❌ FAILED: Vague JD must require human review!")
    else:
        print("✅ PASSED all validation checks")

# Upload files
print("📤 Upload resume (PDF):")
resume_file = list(files.upload().keys())[0]
print("📄 Upload job description (TXT):")
jd_file = list(files.upload().keys())[0]

# Agentic flow
state = {}
state.update(resume_parser_agent(parse_resume(resume_file)))

if not state.get("parsing_error"):
    with open(jd_file, 'r') as f:
        state.update(jd_parser_agent(f.read()))

result = match_agent(state)

final_output = {
    "match_score": float(result["match_score"]),
    "recommendation": result["recommendation"],
    "requires_human": bool(result["requires_human"]),
    "confidence": float(result["confidence"]),
    "reasoning_summary": result["reasoning_summary"]
}

validate_output(resume_file, jd_file, final_output)
print("\n" + "="*50)
print("FINAL OUTPUT (Pitcrew Format):")
print(json.dumps(final_output, indent=2))

📤 Upload resume (PDF):


Saving resume_01_priya_sharma.pdf to resume_01_priya_sharma.pdf
📄 Upload job description (TXT):


Saving jd_01_backend_python_standard.txt to jd_01_backend_python_standard.txt
📝 Resume LLM Response: ```json
{
  "name": "Priya Sharma",
  "years_experience": 3,
  "skills": [
    "Python",
    "SQL",
    "JavaScript",
    "Bash",
    "Django",
    "Django REST Framework",
    "FastAPI",
    "Flask",
    "Celery",
    "PostgreSQL",
    "MySQL",
    "Redis",
    "MongoDB",
    "Docker",
    "Git",
 
📝 JD LLM Response: ```json
{
  "role": "Backend Engineer - Python",
  "required_skills": [
    "Python",
    "REST API design",
    "SQL",
    "Git",
    "Software development best practices",
    "Good communication skills in English"
  ],
  "required_experience_years": 2,
  "strictness_level": "strict"
}
```

🔍 VALIDATION: resume_01_priya_sharma.pdf + jd_01_backend_python_standard.txt
✅ Match Score: 0.8
✅ Requires Human: True
✅ Reasoning: Match score 0.8. Matched skills: ['rest api design', 'sql', 'python', 'git']. Experience: 3y vs 2y required. Strictness:...
✅ PASSED all validation check

In [22]:
from google.colab import files

def validate_output(resume_name: str, jd_name: str, result: dict):
    print(f"\n🔍 VALIDATION: {resume_name} + {jd_name}")
    print(f"✅ Match Score: {result['match_score']}")
    print(f"✅ Requires Human: {result['requires_human']}")
    print(f"✅ Reasoning: {result['reasoning_summary'][:120]}...")

    if "vague" in jd_name and not result["requires_human"]:
        print("❌ FAILED: Vague JD must require human review!")
    else:
        print("✅ PASSED all validation checks")

# Upload files
print("📤 Upload resume (PDF):")
resume_file = list(files.upload().keys())[0]
print("📄 Upload job description (TXT):")
jd_file = list(files.upload().keys())[0]

# Agentic flow
state = {}
state.update(resume_parser_agent(parse_resume(resume_file)))

if not state.get("parsing_error"):
    with open(jd_file, 'r') as f:
        state.update(jd_parser_agent(f.read()))

result = match_agent(state)

final_output = {
    "match_score": float(result["match_score"]),
    "recommendation": result["recommendation"],
    "requires_human": bool(result["requires_human"]),
    "confidence": float(result["confidence"]),
    "reasoning_summary": result["reasoning_summary"]
}

validate_output(resume_file, jd_file, final_output)
print("\n" + "="*50)
print("FINAL OUTPUT (Pitcrew Format):")
print(json.dumps(final_output, indent=2))

📤 Upload resume (PDF):


Saving resume_03_ananya_patel.pdf to resume_03_ananya_patel (1).pdf
📄 Upload job description (TXT):


Saving jd_03_junior_flexible.txt to jd_03_junior_flexible (1).txt
📝 Resume LLM Response: ```json
{
  "name": "Ananya Patel",
  "years_experience": 2,
  "skills": [
    "JavaScript",
    "TypeScript",
    "HTML5",
    "CSS3",
    "React.js",
    "Next.js",
    "Vue.js",
    "Tailwind CSS",
    "SCSS",
    "Git",
    "Webpack",
    "Vite",
    "Figma",
    "VS Code",
    "Chrome DevTools"
📝 JD LLM Response: ```json
{
  "role": "Junior Backend Developer",
  "required_skills": [
    "Basic understanding of Python programming",
    "Some exposure to web development concepts (APIs, HTTP, databases)",
    "Good problem-solving attitude",
    "Ability to communicate in English"
  ],
  "required_experience_ye

🔍 VALIDATION: resume_03_ananya_patel (1).pdf + jd_03_junior_flexible (1).txt
✅ Match Score: 0.28
✅ Requires Human: False
✅ Reasoning: Match score 0.28. Matched skills: []. Experience: 2y vs 0y required. Strictness: flexible....
✅ PASSED all validation checks

FINAL OUTPUT (Pitcrew Format)

In [23]:
from google.colab import files

def validate_output(resume_name: str, jd_name: str, result: dict):
    print(f"\n🔍 VALIDATION: {resume_name} + {jd_name}")
    print(f"✅ Match Score: {result['match_score']}")
    print(f"✅ Requires Human: {result['requires_human']}")
    print(f"✅ Reasoning: {result['reasoning_summary'][:120]}...")

    if "vague" in jd_name and not result["requires_human"]:
        print("❌ FAILED: Vague JD must require human review!")
    else:
        print("✅ PASSED all validation checks")

# Upload files
print("📤 Upload resume (PDF):")
resume_file = list(files.upload().keys())[0]
print("📄 Upload job description (TXT):")
jd_file = list(files.upload().keys())[0]

# Agentic flow
state = {}
state.update(resume_parser_agent(parse_resume(resume_file)))

if not state.get("parsing_error"):
    with open(jd_file, 'r') as f:
        state.update(jd_parser_agent(f.read()))

result = match_agent(state)

final_output = {
    "match_score": float(result["match_score"]),
    "recommendation": result["recommendation"],
    "requires_human": bool(result["requires_human"]),
    "confidence": float(result["confidence"]),
    "reasoning_summary": result["reasoning_summary"]
}

validate_output(resume_file, jd_file, final_output)
print("\n" + "="*50)
print("FINAL OUTPUT (Pitcrew Format):")
print(json.dumps(final_output, indent=2))

📤 Upload resume (PDF):


Saving resume_04_vikram_singh.pdf to resume_04_vikram_singh (1).pdf
📄 Upload job description (TXT):


Saving jd_04_vague_ambiguous.txt to jd_04_vague_ambiguous (1).txt
📝 Resume LLM Response: ```json
{
  "name": "VIKRAM SINGH",
  "years_experience": 8,
  "skills": [
    "Python",
    "SQL",
    "JavaScript",
    "Django",
    "Django REST Framework",
    "Flask",
    "PostgreSQL",
    "SQLite",
    "Git",
    "GitHub",
    "VS Code",
    "Postman",
    "Docker",
    "AWS basics",
    "Re
📝 JD LLM Response: ```json
{
  "role": "Software Developer",
  "required_skills": [
    "Programming knowledge",
    "Database experience",
    "Coding skills",
    "Some framework experience",
    "Database"
  ],
  "required_experience_years": 0,
  "strictness_level": "vague"
}
```

🔍 VALIDATION: resume_04_vikram_singh (1).pdf + jd_04_vague_ambiguous (1).txt
✅ Match Score: 0.28
✅ Requires Human: True
✅ Reasoning: Match score 0.28. Matched skills: []. Experience: 8y vs 0y required. Strictness: vague....
✅ PASSED all validation checks

FINAL OUTPUT (Pitcrew Format):
{
  "match_score": 0.28,
  "recommenda